In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Load lenti data for all cell types (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'lenti')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data[(data['construct'].isin(['UI','UT'])) | (data['virus_dilution']==0)].groupby(['cell','dox','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs
annotate_kwargs = base.annotate_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['full'], 1.5*5))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 6, figure=fig, wspace=0.4, hspace=0.4, 
                                            height_ratios=[1.5]*5, width_ratios=[1.2,0.8,1,1,0.9,1.1])

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:1]),
    'B': fig.add_subfigure(fig_gridspec[0,1:]),
    'C': fig.add_subfigure(fig_gridspec[1,:5]),
    'D': fig.add_subfigure(fig_gridspec[1,5]),
    'E': fig.add_subfigure(fig_gridspec[2,:4]),
    '': fig.add_subfigure(fig_gridspec[2,4:]),
    'F': fig.add_subfigure(fig_gridspec[3,:5]),
    'G': fig.add_subfigure(fig_gridspec[3,5]),
    'H': fig.add_subfigure(fig_gridspec[4,:5]),
    'I': fig.add_subfigure(fig_gridspec[4,5]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_lenti-supp'/'fig_lenti-supp.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.1)
ax = subfig.subplots(1,1)

plot_df = stats[(stats['group'].isin(['base','controller'])) & (stats['cell']=='neuron') & (stats['dox']==1000) & (stats['moi']==7) &
                ~(stats['name'].str.contains('FXN')) & ~(stats['name'].str.contains('FMRP'))]
xlim = (-0.5, len(plot_df['ts_label'].unique())-0.5)
pairs = [('CL','base'), ('CL','OL')]

# std
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e2,7e4))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)
ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,2, figsize=(4,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_std', 'output_variation']
for ax, stat in zip(axes, stat_list):      
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    if stat == 'output_std': ax.set(yscale='log',)
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()      

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.12)
axes = subfig.subplots(1,4, gridspec_kw=dict(width_ratios=(1,1,1,1.5), wspace=0.4))

plot_df = stats[(stats['cell']=='MEF') & (stats['dox']==1000) & (stats['moi']==1) & 
                (stats['group'].isin(['base','controller']))]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='design', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='design', ylabel='', yscale='log', ylim=(3e1,1e4))
ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])
baseline = baseline_df.loc[(baseline_df['cell']=='MEF'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='design', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(3e1,1e4))
ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='design', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0,1.1))
ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])

# CDF
ax = axes[3]
plot_df2 = quantiles[(quantiles['cell']=='MEF') & (quantiles['dox']==1000) & (quantiles['moi']==1) & 
                (quantiles['group'].isin(['base','controller'])) & quantiles['biorep']==1]
plot_df2.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df2['construct'].unique())

sns.kdeplot(data=plot_df2, x='output', hue='construct', palette=main_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(5e0,4e4), title='CDF')
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
pairs = [('base','CL'), ('OL','CL')]
designs = plot_df['design'].unique()
for stat in ['output_gmean', 'output_std', 'slope', 'output_variation']: 
    f, axes = plt.subplots(1,len(designs)-1, figsize=(5,2), gridspec_kw=dict(wspace=0.5))
    print('----------- '+stat+' -----------')

    for ax, design in zip(axes, designs[1:]):
        print('Design: '+str(design))
        test_df = plot_df[(plot_df['design']==design) | (plot_df['group']=='base')]
        sns.stripplot(data=test_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False)
        ax.set(ylabel='', xlabel='', title=str(design),)
        if stat is 'output_gmean' or stat is 'output_std': ax.set(yscale='log')
        annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y=stat)
        annotator.configure(**annotate_kwargs,).apply_and_annotate()
        print('\n')
    axes[0].set(ylabel=stat)

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,1, gridspec_kw=dict(wspace=0.4))

plot_df = stats[(stats['group'].isin(['base','controller'])) & (stats['cell']=='MEF') & (stats['dox']==0) & (stats['moi']==1)]
xlim = (-0.5, len(plot_df['design'].unique())-0.5)

# stat mean
ax = axes
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='design', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='design', ylabel='', yscale='log', ylim=(1e1,1e2))

ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])
baseline = baseline_df.loc[(baseline_df['cell']=='MEF'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.35, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.3))

# lenti 293T controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats[~stats['group'].isin(['controller','marker']) & (stats['cell']=='MEF') &
                      (stats['dox']==1000) & (stats['moi']==1)].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
num_groups = 3
xtick_locs = [0, 1+buffer, 2+buffer] + [i+buffer*2 for i in range(3,7)]
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
xlim = (-0.5, plot_df['construct_loc'].max()+0.5)

# adjust markers
m.loc[(m['miR_loc']=='CDS') | (m['ts_loc']=='3\''), 'markers'] = 'D'
m_dict = m.set_index('construct').to_dict('dict')
comb_markers = m_dict['markers']

# make xticklabels
def get_label(df):
    group = df['group'].unique()[0]
    d = df.copy()
    col_map = {'base': 'group', 'miR': 'miR', 'ts3': 'ts', 'ts5': 'ts'}
    d['label'] = d[col_map[group]]
    return d

m = m.groupby('group')[m.columns].apply(get_label).reset_index(drop=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs, ylim=(3e1,1.5e4))
baseline = baseline_df.loc[(baseline_df['cell']=='MEF'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs, ylim=(3e1,1.5e4))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', xticks=xtick_locs, ylim=(0.5,1.3))

# add shaded region for miR-only constructs
for ax in axes:
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    labels = dict(zip(m.drop_duplicates('construct_loc')['construct_loc'], m.drop_duplicates('construct_loc')['label']))
    ax.set_xticklabels([labels[l.get_text()].replace('.','-') for l in ax.get_xticklabels()], ha='right', rotation=45)
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
construct_base = plot_df.loc[plot_df['group']=='base', 'construct'].values[0]
pairs = [(construct_base,c) for c in plot_df['construct'].unique() if c != construct_base]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='construct', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    labels = dict(zip(m.drop_duplicates('construct_loc')['construct'], m.drop_duplicates('construct_loc')['label']))
    ax.set_xticklabels([labels[l.get_text()].replace('.','-') for l in ax.get_xticklabels()], ha='right', rotation=45)
    if stat != 'slope': ax.set(yscale='log',)
    annotator = Annotator(ax, pairs, data=plot_df, x='construct', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.12)
axes = subfig.subplots(1,4, gridspec_kw=dict(width_ratios=(1,1,1,1.5), wspace=0.4))

plot_df = stats[(stats['cell']=='tcell') & (stats['dox']==1000) & (stats['moi']==1)]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e2,5e6), yticks=(1e2,1e3,1e4,1e5,1e6))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
baseline = baseline_df.loc[(baseline_df['cell']=='tcell'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(2e4,2e6))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.5,2.3))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CDF
ax = axes[3]
plot_df2 = quantiles[(quantiles['cell']=='tcell') & (quantiles['dox']==1000) & (quantiles['moi']==1) & (quantiles['biorep']==1)]
plot_df2.sort_values(['ts_num','ts_label'], ascending=False, inplace=True)
plot_order = reversed(plot_df2['construct'].unique())

sns.kdeplot(data=plot_df2, x='output', hue='construct', palette=main_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(4e2,4e6), title='CDF')
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,4, figsize=(8,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope', 'output_variation']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    if stat is 'output_gmean' or stat is 'output_std': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,1, gridspec_kw=dict(wspace=0.4))

plot_df = stats[(stats['cell']=='tcell') & (stats['dox']==0) & (stats['moi']==1)]
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)

# stat mean
ax = axes
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e1,8e3))

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)
baseline = baseline_df.loc[(baseline_df['cell']=='tcell'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.12)
axes = subfig.subplots(1,4, gridspec_kw=dict(width_ratios=(1,1,1,1.5), wspace=0.4))

plot_df = stats[(stats['cell']=='iPS11') & (stats['dox']==1000) & (stats['moi']==1)]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(3e1,7e5), yticks=(1e2,1e3,1e4,1e5))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
baseline = baseline_df.loc[(baseline_df['cell']=='iPS11'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(1e3,2e5))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.5,2))
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CDF
ax = axes[3]
plot_df2 = quantiles[(quantiles['cell']=='iPS11') & (quantiles['dox']==1000) & (quantiles['moi']==1) & (quantiles['biorep']==1)]
plot_df2.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df2['construct'].unique())

sns.kdeplot(data=plot_df2, x='output', hue='construct', palette=main_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(4e2,2e5), title='CDF')
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,4, figsize=(8,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope', 'output_variation']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    if stat is 'output_gmean' or stat is 'output_std': ax.set(yscale='log')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,1, gridspec_kw=dict(wspace=0.4))

plot_df = stats[(stats['cell']=='iPS11') & (stats['dox']==0) & (stats['moi']==1)]
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)

# stat mean
ax = axes
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e1,1.5e2))

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)
baseline = baseline_df.loc[(baseline_df['cell']=='iPS11'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_lenti-supp.pdf'))

In [ ]:
f, axes = plt.subplots(1,5, figsize=(10,2), gridspec_kw=dict(wspace=0.4))
for ax, cell in zip(axes, ['293T','neuron','MEF','tcell','iPS11']):
    plot_df = stats[(stats['cell']==cell) & (stats['design']<=1) & (stats['group'].isin(['base','controller'])) & (stats['dox']==1000)]
    if cell == 'neuron': plot_df = plot_df[plot_df['moi']==7]
    else: plot_df = plot_df[plot_df['moi']==1]

    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=main_palette,
                    ax=ax, marker=main_markers[construct], legend=False)
    ax.set(title=cell, xlim=xlim, xlabel='', ylabel='', )#yscale='log', ylim=(1e1,2e2))

f.savefig(rd.outfile(rd.rootdir/'output'/'fig_lenti-supp'/'CV.png'))